In [1]:
# System 
#This script computes the HWMI based on daily fields for a specific period in cross-validation. 
# import matplotlib
# from scipy import signal
# import os
# import datetime
# import copy
# import shutil
# import sys
# from function_read import *
# from scipy.stats import gaussian_kde
# import calendar
# import locale
# from netCDF4 import num2date, date2num
# from netCDF4 import num2date, date2num, Dataset
# from numpy import dtype

In [2]:
import numpy as np
import xarray as xr
from joblib import Parallel, delayed
import joblib
import time

In [3]:
var='t2m'
expname='ERA5'
memb_str='0' # there are not members for ERA5
nrealisation=1

reg_name ='test'

# Region #
if reg_name == 'test':
    min_lon = 10
    min_lat = 40
    max_lon = 20
    max_lat = 50

if reg_name == 'Europe':
    min_lon = -15
    min_lat = 25
    max_lon = 60
    max_lat = 70
if reg_name == 'global':
    min_lon = 0
    min_lat = -90
    max_lon = 360
    max_lat = 90

lats_bnds = np.array([min_lat,max_lat])
lons_bnds = np.array([min_lon, max_lon])
# Season #
season='15MJJA'
season_start_day=[5,15]
season_start_day=[8,31]

# nday=109 #15 of May to 31st of Aug
nday=365
#nday = 109*3
# Period #
ref_year1=2015
ref_year2=2017
nyear=len(range(ref_year1,ref_year2))+1

In [4]:
low=False # if data with 1 degree resolution is used (True) or not (False)

if (low==True):
    # path="/work/csp/vt17420/ERA5/daily/russo/"+var+"_r360x180/"
    # Where the outputs will be saved
    # dirout="/work/csp/vt17420/ERA5/daily/russo/"+var+"_r360x180/HWMI/"
    path="../testdata/"
    dirout="../testdata/"
    lons_reg=np.arange(min_lon,max_lon,1)
    lats_reg=np.arange(min_lat,max_lat,1)
    nlon=len(lons_reg)
    nlat=len(lats_reg)
else:
    # path="/work/csp/vt17420/ERA5/daily/global/"+var+"/"
    # dirout="/data/csp/vt17420/CLINT_PRODUCTS/"+expname+"_HWMI_"+var+"/"
    path="../testdata/"
    dirout="../testdata/"
    lons_reg=np.arange(min_lon,max_lon+0.25,0.25)
    lats_reg=np.arange(min_lat,max_lat+0.25,0.25)
    nlon=len(lons_reg)
    nlat=len(lats_reg)

In [5]:
# file preaparation and data selections. 
file = '../testdata/download_2015-2017.nc'
# nlon = 241 

data=np.zeros((nlon,nlat,nyear,nday,1)) # daily data for all the years
# data=np.zeros((241,181,nyear,nday,1))
# data.shape

In [6]:
for iyear,year in enumerate(range(ref_year1,ref_year2+1)):
    print(year)
    print(iyear)
    days_may=np.linspace(15, 31, num=17)
    # obs1=xr.open_dataset(path+var+"_"+str(year)+'05.nc')
    obs1=xr.open_dataset(file)
    if low==False:
    # selecting BBox:    
        obs1 = obs1.sel(latitude=slice(max_lat,min_lat),longitude=slice(min_lon,max_lon))
    # selecting time and bbox 
    obs=obs1.sel(time=obs1.time.time.dt.year.isin(year))
    obs = obs.sel(time=~((obs.time.dt.month == 2) & (obs.time.dt.day == 29)))
    # selection of the days in june, july and august
#     for mon in range(6,9):
#         # obs1=xr.open_dataset(path+var+"_"+str(year)+str(mon).zfill(2)+'.nc')
#         obs1=xr.open_dataset(file)
#         if low==False:
#             obs1 = obs1.sel(latitude=slice(max_lat,min_lat),longitude=slice(min_lon,max_lon))
#         obs=xr.merge(([obs,obs1]))
#    data[:,:,iyear,:,0]=np.transpose(obs.to_array()[0,:,:,:],[2,1,0])
#    obs.shape()
    data[:,:,iyear,:,0]=np.transpose(obs.to_array()[0,:,:,:],[2,1,0])

2015
0
2016
1
2017
2


In [2]:
slice(max_lat,min_lat)
slice(min_lon,max_lon)

NameError: name 'max_lat' is not defined

In [8]:
### PERCENTILE THRESHOLD
#percent_thresh = 95
percent_thresh = 90
cv='CV'
cv_str = cv
### MINIMAL DURATION OF A HW
#duration_min = 5
duration_min = 3

In [9]:
from HWMI import *

In [10]:
nmemb=data.shape[4]
ndayseas = nday//duration_min +1
HWMI = np.zeros((nyear,nmemb,nlat,nlon))
ndayexedthreshold = np.zeros((nyear,nmemb,nlat,nlon))
subHWarray = np.zeros((nyear, ndayseas, nmemb, nlat, nlon))
#HW = np.zeros((nyear,nmemb,nday,nlat,nlon))
DDthreshold = np.zeros((nyear,nmemb,nlat,nlon))
fitsubHWarray = np.zeros((nyear, ndayseas, nmemb, nlat, nlon))
impossible_fit_list = []
#sstMeanarray = np.zeros((nyear,nmemb,nlat,nlon))

def parallelized_HWMIs_computation(ilat, ilon):    #, HWMI, ndayexedthreshold, DDthreshold, fitsubHWarray, subHWarray):
        HWMIyear, HWlstyear, HWstartmembyear, HWendmembyear, ndayexedthresholdyear, DDthresholdyear, subHWarrayyear, fitsubHWarrayyear, sstMeanarrayyear, impossible_fit = calc_HWMIyear(data[ilon,ilat,:,:,:], cross_valid = cv, percent_thresh = percent_thresh, duration_min = duration_min)
        #print(HWMIyear)
        HWMI[:,:,ilat,ilon]=np.array(HWMIyear)
        ndayexedthreshold[:,:,ilat,ilon]=np.array(ndayexedthresholdyear)
        subHWarray[:,:, :, ilat,ilon]=np.array(subHWarrayyear)
        DDthreshold[:,:,ilat,ilon]=np.array(DDthresholdyear)
        fitsubHWarray[:,:, :, ilat,ilon]=np.array(fitsubHWarrayyear)
        #sstMeanarray[:,:,ilat,ilon]=np.array(sstMeanarrayyear)
        impossible_fit_list.append((impossible_fit,ilat,ilon))
        return()

In [11]:
Parallel(n_jobs=-1, timeout = 5*3600, verbose = 20, require='sharedmem', mmap_mode='w+')(delayed(parallelized_HWMIs_computation)(ilat, ilon) for ilat in range(nlat) for ilon in range(nlon))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapse

[(),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),


In [12]:
if os.path.isdir(dirout)==False:
    os.makedirs(dirout)
years=range(ref_year1,ref_year2)
j=0
for i,iyear in enumerate(range(ref_year1, ref_year2+1)):
    #print iyear
    parameters_str = reg_name+"_"+season+"_"+cv_str+'_percent%i'%(percent_thresh)+'_daymin%i'%(duration_min)+"_ref_"+str(ref_year1)+"_"+str(ref_year2)+"_year_"+str(iyear)
    varout1 = "HWMI"+"_"+var+"_"+parameters_str
    vout1="HWMI"+"_"+var
    fileout=dirout+varout1+".nc"#0%i01, monstart)
    print(fileout)
    if len(glob(fileout))==1:
        os.remove(fileout)
    fout=netCDF4.Dataset(fileout, "w")
    #fin=netCDF4.Dataset(targetflst[iyear])
    lat = fout.createDimension('lat', nlat)
    lon = fout.createDimension('lon', nlon) 
    rea = fout.createDimension('realisation', nrealisation) 
    latitudes = fout.createVariable('lat', np.float32, ('lat',)) 
    longitudes = fout.createVariable('lon', np.float32,  ('lon',)) 

    # Time variable
    timedim = fout.createDimension('time', None) 
    times = fout.createVariable('time', np.float64, ('time',)) 
    times.units = "hours since 1970-01-01 00:00:00"
    times.calendar = 'gregorian' 
    times[:]=date2num(datetime(iyear,season_start_day[0],season_start_day[1]), units=times.units,calendar = times.calendar) ####______HERE_____


    latitudes[:] = lats_reg
    lonaux = lons_reg
    longitudes[:] = lonaux
    latitudes.units = 'degree_north'  
    longitudes.units = 'degree_east' 

    # Create the HWMI 4-d variable
    HWMIfile = fout.createVariable(vout1, np.float32, ('time','realisation','lat','lon'))
    fout.description = 'HWMI index (Russo et al. 2014) for ' + season + ' computed in cross validation'
    fout.history = 'computed from python script by C.Prodhomme & S.Lecestre' + time.ctime(time.time())
    fout.source = 'HWMI for ' + expname
    latitudes.units = 'degree_north'  
    longitudes.units = 'degree_east' 
    HWMIfile.units = 'Probability'

    # Create the nb of days 4-d variable
    expercentfile = fout.createVariable("nbdaygtpercentpct_"+var, np.float32, ('time','realisation','lat','lon')) 
    expercentfile.units = 'Number of days'  

    # Write the HWMI variable
    HWMIaux=HWMI[i:i+1,0:1,:,:]
    HWMIfile[0:1,0:1,:,:]=HWMIaux

    # Write the number of days
    exedaux=ndayexedthreshold[i:i+1,j:j+1,:,:]
    expercentfile[0:1,0:1,:,:]=exedaux 

    # Write the DDthreshold
    #DDthresholdfile=DDthreshold
    fout.close()

../testdata/HWMI_t2m_test_15MJJA_CV_percent90_daymin3_ref_2015_2017_year_2015.nc
../testdata/HWMI_t2m_test_15MJJA_CV_percent90_daymin3_ref_2015_2017_year_2016.nc
../testdata/HWMI_t2m_test_15MJJA_CV_percent90_daymin3_ref_2015_2017_year_2017.nc
